## Imports

In [66]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from IPython.display import clear_output
print(tf.__version__)

2.7.0


# Reinforcement learning

Reinforcement learning (RL) is an area of ML related to training an intelligent agent which performs a sequence of decisions. The agent learns to achieve a goal in an uncertain, potentially complex environment. Via trials and errors the agent comes up with a solution to the problem. The agent gets either rewards or penalties for the actions it performs. Its goal is to maximize the total reward.

Ref: https://deepsense.ai/what-is-reinforcement-learning-the-complete-guide/

## Frozen lake environment

Let's consider simple environment and agent acting in it. `gym` toolkit provides collection of environments that can be simulated in RL suitable way. 

(F)rozen lake is field with size 4x4 units. There are (H)oles in the ice scattered around. (S)tart and (G)oal positions are defined.

The grand objective is to train the agent (marked in red) in such a way that it can successfully cross the lake.

Agent gets reward of 1 if it reaches the goal, and 0 otherwise.


> Note: `is_slippery=False` disables randomness in action results (for demo purposes).

In [5]:
import gym

env = gym.make("FrozenLake-v0", is_slippery=False)
observation = env.reset()
env.render()

print()
print('Observation space:', env.observation_space)
print('Action space:', env.action_space)
print('Current observation:', observation)


SFFF
FHFH
FFFH
HFFG

Observation space: Discrete(16)
Action space: Discrete(4)
Current observation: 0


Let's implement simple agent, which randomly wanders around the lake.

In [ ]:
env.reset()
for step in range(10):
  # randomly select next action
  action = env.action_space.sample()

  # agent performs the selected action, and gets feedback from the environment
  state, reward, done, info = env.step(action)  

  print('===============================================')
  print(f'Step: {step}\tAction: {action}\tReward: {reward}\tNew state: {state}\n')
  env.render()
  print()

  if done:
    print('\nEpisode ended')
    break

The essence of RL is to run number of episodes and modify policy in such a way to get maximum cumulative reward.

# Q-learning

Q-learning is one of RL algorithms, it finds optimal policy and is well suited for discrete problems.

"Q" refers to the function that the algorithm computes: the expected (future) rewards for an action taken in a given state.

https://en.wikipedia.org/wiki/Q-learning

Usually the algorithm is implemneted as table of Q values for each state-action. Using the table current state can be mapped to the best next action.

|         | State A | State B | State C |
|---------|--------:|--------:|--------:|
|Action 1 | 0.17    | 0.56    | 12.43   |
|Action 2 | 0.22    | 0.32    | 32.10   |

The essense of the algorithm lies in how the values of the table are changed.

$$
Q^{new}(s_t, a_t) \leftarrow 
\underbrace{ Q(s_t, a_t) }_\textrm{old value}
 + \underbrace{a}_\textrm{learning rate}
\cdot \left(
   \underbrace{r_t}_\textrm{reward} 
   + \underbrace{\gamma}_\textrm{discount factor}
   \cdot \underbrace{\max_a{Q(s_{t+1}, a)}}_\textrm{best future value}
   - \underbrace{ Q(s_t, a_t) }_\textrm{old value} 
\right)
$$

where:
* $a$ - determiates how fast new values are applied to the table:
  * $0$ - values never change;
  * $1$ - old values are overwritten by new values;
* $\gamma$ - determinates how much importance has the expected future reward:
  * $0$ - only currect step reward is considered;
  * $1$ - infinitely long future is expected (not realistic);


Perhaps more readable / understandable formula looks like follows:
$$
Q^{new}(s_t, a_t) \leftarrow 
\left(1 - a \right) \cdot \underbrace{ Q(s_t, a_t) }_\textrm{old value}
 + a \cdot 
 \left(
   \underbrace{r_t}_\textrm{reward} 
   + \underbrace{\gamma}_\textrm{discount factor}
   \cdot \underbrace{\max_a{Q(s_{t+1}, a)}}_\textrm{best future value}   
\right)
$$



In [ ]:
Q = np.zeros(shape=(env.observation_space.n, env.action_space.n))
# untrained Q-table
Q 

In order to facilitate better and faster training, few tricks are usually applied.

## Exploration vs Exploitation
If only the actions with the highest score are selected from the table, the agent very quickly starts to focus on previous succesfull steps (**exploitation**).

While for training purposes and good global solution the **exploration** is needed (let the agent make mistakes and perhaps it will find better solution).

Usually the ratio between exploration and exploitation is parametrized and decays over time.


## Continous reward

Straightforward reward mechanism can be implemented as the reward only for achieving the goal (like in Frozen lake environment, where agent gets reward of $1$ for reaching the goal position, and $0$ otherwise).

However usually it is better to provide at least some feedback (reward) for each step taked by agent, even if the reward is negative. It can be interpreted as resources or time spent for the step. This approach prevents agents from looping around infinetly.

# Code example

First, we need a few utility functions.

In [5]:
def select_action(observation, explore = 0.0):
  if np.random.uniform() < explore:
    # exploring the environment, taking random action
    return env.action_space.sample()
  else:
    # finding best action for current observation
    return np.argmax(Q[observation, :])

def update_table(observation, observation_next, action, reward):
  learning_rate =  0.7
  future_factor = 0.9
  old_value = Q[observation, action]
  new_value = reward + future_factor * np.max(Q[observation_next, :])
  # updating table value according to formula
  Q[observation, action] = old_value + learning_rate * (new_value - old_value)

In [ ]:
# single step demonstration
state = env.reset()
action = select_action(state, explore=1.0)
new_state, reward, done, _ = env.step(action)
reward -= 0.01 # adding small negative reward for perfroming the step (spent time, energy)

update_table(state, new_state, action, reward)

# output of the table
env.render()
Q

Training the model implies running number of episodes and let the agent explore the environment and find best policy.

In [ ]:
explore = 0.9 # reset exploration rate

# how many training episodes we want to run
for episode in range(10000):
  state = env.reset()  

  # how many steps our agent tries to perform in each episode
  for step in range(100):
    action = select_action(state, explore)
    new_state, reward, done, _ = env.step(action)
    reward -= 0.01
    
    update_table(state, new_state, action, reward)
    state = new_state

    if done:
      # episode ended, either win or lose
      break
  
  # after each episode
  explore = explore * 0.999 # decay exploration factor
  explore = max(explore, 0.05) # usually some exploration probability is kept during the training

# trained Q-table
Q

Now when the agent is trained, it can exploit its knowledge.

In [ ]:
state = env.reset()

for step in range(15):
  action = select_action(state)
  state, reward, done, _ = env.step(action)
  env.render()
  print()
  if done:
    if reward > 0:
      print("Win!")
    break

# Task - Train Q-learning model for 8x8 Frozen lake environment

By default, Frozen lake environment includes various map configurations and *is slippery* (there is random chance that agent is not successful in action).


In [10]:
env = gym.make('FrozenLake-v0', map_name="8x8")
env.reset()
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [11]:
# TODO: build Q model
#Q-model
action_size = env.action_space.n
print("Action size: ", action_size)

state_size = env.observation_space.n
print("State size: ", state_size)

Action size:  4
State size:  64


In [19]:
#Q-table
q_table = np.zeros((state_size, action_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [91]:
num_episodes = 10000       # Total episodes
learning_rate = 0.3          # Learning rate
max_steps = 200               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
exploration_rate = 1.0                 # Exploration rate
max_exploration_rate = 1.0             # Exploration probability at start
min_exploration_rate = 0.01            # Minimum exploration probability 
exploration_decay_rate = 0.002            # Exponential decay rate for exploration prob

In [81]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episodes in range(num_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_rate_threshold = random.uniform(0, 1)
        
        ## If this number > greater than exploration_rate --> exploitation (taking the biggest Q value for this state)
        if exp_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
            #print(exp_rate_threshold, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
            
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        q_table[state, action] = q_table[state, action] + learning_rate * (reward + gamma * np.max(q_table[new_state, :]) - q_table[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce exploration_rate (because we need less and less exploration)
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate)*np.exp(-exploration_decay_rate*episodes) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/num_episodes))
print(q_table)

Score over time: 0.3777
[[3.13888009e-02 3.13342964e-02 3.97625670e-02 3.13316105e-02]
 [3.43979295e-02 3.32496903e-02 3.46009885e-02 5.04662746e-02]
 [3.90879787e-02 3.92230429e-02 5.20078302e-02 4.25245260e-02]
 [5.64576131e-02 5.62062835e-02 6.69810379e-02 5.60162154e-02]
 [5.99709881e-02 6.50878780e-02 8.41874461e-02 6.12863304e-02]
 [7.59583477e-02 7.33394235e-02 9.95360388e-02 7.60263536e-02]
 [8.71820322e-02 9.12080887e-02 1.05156196e-01 8.24192232e-02]
 [7.86519158e-02 9.81837819e-02 1.08784135e-01 8.94387783e-02]
 [3.20145564e-02 3.04639041e-02 3.22969246e-02 3.82058081e-02]
 [2.98287467e-02 3.32869736e-02 3.33637054e-02 4.48872024e-02]
 [3.33619314e-02 3.65898012e-02 2.97581473e-02 5.53533085e-02]
 [3.04613150e-02 4.79579318e-02 4.97820969e-02 7.06455237e-02]
 [6.52843508e-02 5.25786623e-02 9.36294097e-02 6.47458932e-02]
 [7.27922656e-02 8.62500955e-02 1.13529303e-01 7.64682862e-02]
 [8.74021518e-02 8.75386213e-02 1.23866772e-01 8.78420893e-02]
 [9.89187988e-02 9.55168667e-02

In [90]:
# watching the agent play

env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode+1)
    time.sleep(1)

    for step in range(max_steps):
         clear_output(wait=True)
         env.render()
         time.sleep(0.3)
        # Take the action (index) that have the maximum expected future reward given that state
         action = np.argmax(q_table[state,:])
         new_state, reward, done, info = env.step(action)
        
         if done:
            clear_output(wait=True)
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 63:
                print("We reached the Goal")
                time.sleep(0.3)
            else:  
                print("We fell in the a hole ")
                time.sleep(0.3)            
            # We print the number of step it took.
            print("Number of steps", step)
            
            break
            state = new_state
env.close()

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
We fell in the a hole 
Number of steps 7
